In [1]:
import tensorflow as tf
import os
from tensorflow import keras
from tensorflow.keras import layers
from keras.models import Sequential
from keras.layers import Dense, Conv2D, MaxPool2D, Flatten
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Model
import numpy as np

from keras.applications.vgg16 import VGG16

img_height=224
img_width=291
batch_size=32

print("keras version: ", keras.__version__)
print("tensorflow version: " ,tf.__version__)

keras version:  2.4.0
tensorflow version:  2.3.0


In [2]:
train_ds=tf.keras.preprocessing.image_dataset_from_directory(
            '../data', labels='inferred', label_mode='int',
            validation_split=0.2, subset='training', seed=123,
#             color_mode='grayscale', 
            image_size=(img_height, img_width),
            batch_size=batch_size)

Found 280 files belonging to 2 classes.
Using 224 files for training.


In [3]:
val_ds=tf.keras.preprocessing.image_dataset_from_directory(
            '../data', labels='inferred', label_mode='int',
            validation_split=0.2, subset='validation', seed=123,
#             color_mode='grayscale', 
            image_size=(img_height, img_width),
            batch_size=batch_size)

Found 280 files belonging to 2 classes.
Using 56 files for validation.


In [4]:
base_model = keras.applications.Xception(weights='imagenet', 
                                        input_shape=(img_height,img_width,3),
                                        include_top=False)
# base_model = keras.applications.VGG16(weights='imagenet', 
#                                         input_shape=(img_height,img_width,3),
#                                         include_top=False)
base_model.trainable = False

inputs=keras.Input(shape=(img_height,img_width,3))

data_augmentation = keras.Sequential(
                    [
                        layers.experimental.preprocessing.RandomFlip("horizontal",
                                                    input_shape=(img_height,img_width,3)),
                        layers.experimental.preprocessing.RandomFlip("vertical"),
              #         layers.experimental.preprocessing.RandomRotation(0.1, input_shape=(img_height,img_width,1))
                        layers.experimental.preprocessing.RandomRotation(0.1)
                   ]
)

x = data_augmentation(inputs)
norm_layer = keras.layers.experimental.preprocessing.Normalization()
mean =np.array([127.5]*3)
var = mean**2

x = norm_layer(x)
norm_layer.set_weights([mean,var])

x = base_model(x,training=False)
x = keras.layers.GlobalAveragePooling2D()(x)
x = keras.layers.Dropout(0.2)(x)
outputs = keras.layers.Dense(1)(x)
model = keras.Model(inputs, outputs)

model.summary()

Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 224, 291, 3)]     0         
_________________________________________________________________
sequential (Sequential)      (None, 224, 291, 3)       0         
_________________________________________________________________
normalization (Normalization (None, 224, 291, 3)       7         
_________________________________________________________________
xception (Functional)        (None, 7, 9, 2048)        20861480  
_________________________________________________________________
global_average_pooling2d (Gl (None, 2048)              0         
_________________________________________________________________
dropout (Dropout)            (None, 2048)              0         
_________________________________________________________________
dense (Dense)                (None, 1)                

In [ ]:
model.compile(optimizer=keras.optimizers.Adam(),
             loss=keras.losses.BinaryCrossentropy(from_logits=True),
             metrics=[keras.metrics.BinaryAccuracy()])

epochs=20
model.fit(train_ds, epochs=epochs, validation_data=val_ds)

Epoch 1/20
7/7 [==============================] - 13s 2s/step - loss: 0.6872 - binary_accuracy: 0.5268 - val_loss: 0.6065 - val_binary_accuracy: 0.9643
Epoch 2/20
7/7 [==============================] - 4s 529ms/step - loss: 0.5201 - binary_accuracy: 0.7857 - val_loss: 0.4879 - val_binary_accuracy: 0.9821
Epoch 3/20
7/7 [==============================] - 4s 507ms/step - loss: 0.4070 - binary_accuracy: 0.8795 - val_loss: 0.3714 - val_binary_accuracy: 1.0000
Epoch 4/20
7/7 [==============================] - 4s 522ms/step - loss: 0.3198 - binary_accuracy: 0.9152 - val_loss: 0.3067 - val_binary_accuracy: 1.0000
Epoch 5/20
7/7 [==============================] - 4s 518ms/step - loss: 0.2615 - binary_accuracy: 0.9732 - val_loss: 0.2711 - val_binary_accuracy: 0.9821
Epoch 6/20
7/7 [==============================] - 4s 527ms/step - loss: 0.2134 - binary_accuracy: 0.9509 - val_loss: 0.2218 - val_binary_accuracy: 1.0000
Epoch 7/20
7/7 [==============================] - 4s 531ms/step - loss: 0.1920

In [11]:
base_model.trainable=True

model.summary()

model.compile(optimizer=keras.optimizers.Adam(1e-5), loss=keras.losses.BinaryCrossentropy(from_logits=True),
             metrics=[keras.metrics.BinaryAccuracy()])

epochs=20
model.fit(train_ds, epochs=epochs, validation_data=val_ds)

Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_6 (InputLayer)         [(None, 224, 291, 3)]     0         
_________________________________________________________________
sequential_1 (Sequential)    (None, 224, 291, 3)       0         
_________________________________________________________________
normalization (Normalization (None, 224, 291, 3)       7         
_________________________________________________________________
vgg16 (Functional)           (None, 7, 9, 512)         14714688  
_________________________________________________________________
global_average_pooling2d (Gl (None, 512)               0         
_________________________________________________________________
dropout (Dropout)            (None, 512)               0         
_________________________________________________________________
dense (Dense)                (None, 1)                